In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/config.json
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/merges.txt
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/README.md
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/vocab.json
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/tokenizer_config.json
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/pytorch_model.bin
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/special_tokens_map.json
/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1/flax_model.msgpack
/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips

In [2]:
train = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
test = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")

In [3]:
import pandas as pd
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

In [4]:
train["SMILES"] = train["SMILES"].fillna("UNKNOWN").astype(str)
test["SMILES"] = test["SMILES"].fillna("UNKNOWN").astype(str)


target_columns = ["Tg", "FFV", "Tc", "Density", "Rg"]
train[target_columns] = train[target_columns].fillna(train[target_columns].median())

In [5]:
from transformers import AutoTokenizer, AutoModel
model_dir = "/kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
bert = AutoModel.from_pretrained(model_dir)

2025-08-31 04:51:28.101087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756615888.480681      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756615888.585176      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
class PolymerDataset(Dataset):
    def __init__(self, smiles, targets=None, max_length=256):
        self.smiles = smiles
        self.targets = targets
        self.max_length = max_length

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        smiles = self.smiles[idx]
        
        # Tokenize the SMILES string
        inputs = tokenizer(smiles, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        inputs = {key: value.squeeze(0) for key, value in inputs.items()}
        
        if self.targets is not None:
            target = self.targets[idx]
            return inputs, torch.tensor(target, dtype=torch.float32)
        else:
            return inputs

In [8]:
X_train = train["SMILES"].values
y_train = train[target_columns].values
X_test = test["SMILES"].values

train_dataset = PolymerDataset(X_train, y_train)
test_dataset = PolymerDataset(X_test, targets=None)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/chemberta-zinc-base-v1.0/transformers/default/1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(767, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [11]:
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros((len(X_train), 5))  # Out-of-fold predictions
test_preds = np.zeros((len(X_test), 5))  # Test predictions

In [13]:
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\nTraining for Fold {fold + 1}/5")

    # Split data for training and validation
    X_train_fold = X_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_train_fold = y_train[train_idx]
    y_val_fold = y_train[val_idx]

    train_dataset = PolymerDataset(X_train_fold, y_train_fold)
    val_dataset = PolymerDataset(X_val_fold, y_val_fold)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    # Initialize the best validation loss and patience counter
    best_val_loss = float("inf")
    patience_counter = 0

    # Training loop for the fold
    for epoch in range(10):  # Set a maximum number of epochs
        model.train()
        train_loss = 0

        # Training step
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} - Training"):
            inputs, targets = batch
            inputs = {key: value.to(device) for key, value in inputs.items()}
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(**inputs).logits.squeeze(-1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}")

        # Validation step
        model.eval()
        val_loss = 0
        val_preds = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch + 1} - Validation"):
                inputs, targets = batch
                inputs = {key: value.to(device) for key, value in inputs.items()}
                targets = targets.to(device)

                outputs = model(**inputs).logits.squeeze(-1)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                val_preds.extend(outputs.cpu().numpy())

        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1} - Validation Loss: {val_loss:.4f}")

        # Learning rate scheduler
        scheduler.step(val_loss)

        # Early stopping: Check if validation loss improved
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 3:  # Patience for early stopping
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    # Store out-of-fold predictions
    oof_preds[val_idx, :] = np.array(val_preds)

    # Make predictions on the test set for the current fold
    model.eval()
    test_fold_preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Fold {fold + 1} - Test Predictions"):
            inputs = batch
            inputs = {key: value.to(device) for key, value in inputs.items()}
            outputs = model(**inputs).logits.squeeze(-1)
            test_fold_preds.extend(outputs.cpu().numpy())

    test_preds[:, :] += np.array(test_fold_preds) / 5  # Average predictions across folds


Training for Fold 1/5


Epoch 1 - Training: 100%|██████████| 399/399 [02:18<00:00,  2.89it/s]


Epoch 1 - Train Loss: 1043.7367


Epoch 1 - Validation: 100%|██████████| 100/100 [00:11<00:00,  9.03it/s]


Epoch 1 - Validation Loss: 851.6141


...

Epoch 10 - Training: 100%|██████████| 399/399 [02:25<00:00,  2.75it/s]


Epoch 10 - Train Loss: 338.8767


Epoch 10 - Validation: 100%|██████████| 100/100 [00:11<00:00,  9.01it/s]


Epoch 10 - Validation Loss: 320.5195


Fold 5 - Test Predictions: 100%|██████████| 1/1 [00:00<00:00, 42.93it/s]


In [14]:
for target_idx, target in enumerate(target_columns):
    mae = mean_absolute_error(y_train[:, target_idx], oof_preds[:, target_idx])
    print(f"MAE for {target}: {mae:.4f}")

MAE for Tg: 32.7425
MAE for FFV: 0.0189
MAE for Tc: 0.0109
MAE for Density: 0.0111
MAE for Rg: 0.3880


In [15]:
submission = pd.DataFrame(test_preds, columns=target_columns)
submission["id"] = test["id"]
submission.to_csv("submission.csv", index=False)
print("Submission file created.")

Submission file created.
